In [1]:
import carla
import math
import random

In [2]:
client = carla.Client('localhost', 2000)
world = client.get_world()
world_map = world.get_map()
spawn_points = world_map.get_spawn_points()
start_point = spawn_points[0]

In [3]:
# Get the blueprint of a black car 
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.toyota.prius'))
vehicle_bp.set_attribute('color', '0,0,0')


In [4]:
# Spawn a vehicle on where I am looking at

# first get where I am looking at
spectator = world.get_spectator()
spectator_transform = spectator.get_transform()
location = spectator_transform.location
rotation = spectator_transform.rotation


distance_ahead = 10  # Spawn the car 10 meters in front
yaw = math.radians(rotation.yaw)  # Convert yaw to radians

spawn_location = carla.Location(
    x=location.x + distance_ahead * math.cos(yaw),
    y=location.y + distance_ahead * math.sin(yaw),
    z=location.z  # Keep the Z-height same (adjust if needed)
)

vehicle = world.try_spawn_actor(vehicle_bp, start_point)
spawn_transform = carla.Transform(spawn_location, rotation)

# Spawn the vehicle
jake = world.try_spawn_actor(vehicle_bp, spawn_transform)
# Spawn a vehicle on where I am looking at

In [5]:
import cv2
import numpy as np
import threading
from yolo import *

# Get YOLO Segment model
model = get_model()

# create the BEV camera above the parking lot 
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '800')
camera_bp.set_attribute('image_size_y', '600')
camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed

bev_location = carla.Location(x=-58.686981, y=166.857986, z=24.152607) 
spectator_rotation = carla.Rotation(pitch=-90, yaw=-180, roll=0) 

# Create a transform using the given location and rotation
bev_transform = carla.Transform(bev_location, spectator_rotation)
# Spawn the camera at the spectator's position and orientation
camera = world.spawn_actor(camera_bp, bev_transform)

def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
    array = array[:, :, :3]  # Remove alpha channel
    results = model(array)
    annotated_frame = results[0].plot()
    cv2.imshow("YOLO Segmentation output", annotated_frame)
    # cv2.imshow("BEV", array)
    cv2.waitKey(1)
    
def display_camera_feed():
    camera.listen(lambda image: process_image(image))

# Start the display in a separate thread so it doesn't block the whole jupyter notebook
camera_thread = threading.Thread(target=display_camera_feed)
camera_thread.daemon = True  # Daemon thread will exit when the main program ends
camera_thread.start()


0: 480x640 1 cell phone, 319.3ms
Speed: 312.0ms preprocess, 319.3ms inference, 31.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cell phones, 64.5ms
Speed: 27.5ms preprocess, 64.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cell phone, 53.6ms
Speed: 6.5ms preprocess, 53.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cell phones, 60.1ms
Speed: 5.1ms preprocess, 60.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cell phone, 61.6ms
Speed: 5.5ms preprocess, 61.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cell phones, 54.2ms
Speed: 5.5ms preprocess, 54.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cell phone, 54.3ms
Speed: 4.5ms preprocess, 54.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 cell phone, 51.7ms
Speed: 5.5ms preprocess, 51.7ms inference, 1.0ms

In [ ]:
jake.set_autopilot(True)
